<a href="https://colab.research.google.com/github/LilySaya/Fundamentals_of_AI/blob/main/CNN_with_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.auto import tqdm

# GPUが使える場合は使う
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"device: {device}")


device: cuda:0


In [ ]:
#hyper parameters
num_epochs = 150
learning_rate = 0.001

In [ ]:
# MNISTのデータをダウンロード（初回のみ）＆読み込み
from torchvision import datasets, transforms

# 画像に対する変換（正規化処理）を定義する
# データ読み込み時に自動的にこの処理が適用される
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])

# 訓練データセットと検証データセットを読み込む
batch_size=512
trainset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
validset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=False, num_workers=2)

# trainset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
# validset = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)
# validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=False, num_workers=2)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# データをいくつか図示
fig, ax = plt.subplots(2, 5, figsize=(15, 6), sharex='col', sharey='row')
for i in range(2):
    for j in range(5):
        ax[i,j].imshow(trainset.data[i*5+j].reshape(28,28), cmap=plt.cm.gray_r)

# plt.savefig("digits.png")

In [ ]:
kernel1 = np.array([[-1, -1, -1],
                    [ 2,  2,  2],
                    [-1, -1, -1]],
                  np.float32)
kernel2 = np.array([[-1, 2, -1],
                    [-1, 2, -1],
                    [-1, 2, -1]],
                  np.float32)
kernel3 = np.array([[-1, -1, -1],
                    [-1,  8, -1],
                    [-1, -1, -1]],
                  np.float32)
kernel1 = torch.as_tensor(kernel1.reshape(1, 1, 3, 3))
kernel2 = torch.as_tensor(kernel2.reshape(1, 1, 3, 3))
kernel3 = torch.as_tensor(kernel3.reshape(1, 1, 3, 3))

image = trainset.data[2].reshape(1, 1, 28, 28).to(torch.float)

fimage1 = F.conv2d(image, kernel1)
fimage2 = F.conv2d(image, kernel2)
fimage3 = F.conv2d(image, kernel3)

fig, ax = plt.subplots(1, 4, figsize=(20, 5), sharex=True, sharey=True)
ax[0].imshow(image[0, 0, :, :], cmap=plt.cm.gray_r)
ax[0].set_title('Input')
ax[1].imshow(fimage1[0, 0, :, :], cmap=plt.cm.gray_r)
ax[1].set_title('Horizontal filter')
ax[2].imshow(fimage2[0, 0, :, :], cmap=plt.cm.gray_r)
ax[2].set_title('Vertical filter')
ax[3].imshow(fimage3[0, 0, :, :], cmap=plt.cm.gray_r)
ax[3].set_title('Edge filter')

# plt.savefig('filters.png', dpi=300)


In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(images.shape)
print(labels.unique().shape)
print(len(trainloader))


torch.Size([512, 1, 28, 28])
torch.Size([10])
118


In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()  #512,1,28,28        #512,16,11,11
        self.conv1 = nn.Conv2d(1, 16, 5)  #512,16,24,24 conv1  #512,32,4,4 pool2
        self.pool = nn.MaxPool2d(2, 2)   #512,16,12,12 pool1
        self.conv2 = nn.Conv2d(16, 32, 5) #512,32,8,8 conv2
        self.fc1 = nn.Linear(32*4*4, 100)
        self.fc2 = nn.Linear(100, 40)
        self.fc3 = nn.Linear(40, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 4 * 4)
        #x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(trainloader) #num_of_batchess
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch+1)%20 == 0:
          if (i+1) % 12 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
#PATH = './cnn.pth'
#torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in validloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()


    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')


Epoch [20/150], Step [12/118], Loss: 2.2767
Epoch [20/150], Step [24/118], Loss: 2.2764
Epoch [20/150], Step [36/118], Loss: 2.2746
Epoch [20/150], Step [48/118], Loss: 2.2765
Epoch [20/150], Step [60/118], Loss: 2.2807
Epoch [20/150], Step [72/118], Loss: 2.2793
Epoch [20/150], Step [84/118], Loss: 2.2838
Epoch [20/150], Step [96/118], Loss: 2.2770
Epoch [20/150], Step [108/118], Loss: 2.2710
Epoch [40/150], Step [12/118], Loss: 2.1345
Epoch [40/150], Step [24/118], Loss: 2.1250
Epoch [40/150], Step [36/118], Loss: 2.1310
Epoch [40/150], Step [48/118], Loss: 2.1062
Epoch [40/150], Step [60/118], Loss: 2.1229
Epoch [40/150], Step [72/118], Loss: 2.1043
Epoch [40/150], Step [84/118], Loss: 2.1107
Epoch [40/150], Step [96/118], Loss: 2.1045
Epoch [40/150], Step [108/118], Loss: 2.1023
Epoch [60/150], Step [12/118], Loss: 0.9059
Epoch [60/150], Step [24/118], Loss: 0.8740
Epoch [60/150], Step [36/118], Loss: 0.8934
Epoch [60/150], Step [48/118], Loss: 0.9066
Epoch [60/150], Step [60/118],

In [ ]:
from google.colab import files
import re
uploaded = files.upload()
for fn in uploaded.keys():
    fn_s = re.escape(fn)
    output_fn = fn.split('.', 1)[0]+'.html'
    output_fn_s = re.escape(output_fn)
    !jupyter nbconvert --to html $fn_s
    files.download(output_fn)
    !rm $fn_s

Saving 20B11806_5.ipynb to 20B11806_5.ipynb
[NbConvertApp] Converting notebook 20B11806_5.ipynb to html
[NbConvertApp] Writing 347359 bytes to 20B11806_5.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>